In [1]:
import os
import dask.dataframe as dd
import pandas as pd
from dask.distributed import Client

In [2]:
client = Client(n_workers=4)
client

Client Scheduler: tcp://127.0.0.1:36878 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 36 Memory: 21.47 GB


In [3]:
os.getcwd()
os.chdir("/data/leuven/351/vsc35102/scratch")

In [59]:
#Get the data
#gdown.download_folder("https://drive.google.com/drive/folders/1HT-ctj8Aj6qcVMZYBxi3YM4XC9fbFjSN?usp=share_link", quiet=False)

In [52]:
def read_csv_files(directory):
    """
    Reads all CSV files in the specified directory into a Dask DataFrame.

    Parameters:
        directory (str): The directory path where the CSV files are located.

    Returns:
        A Dask DataFrame containing all the CSV files in the directory.
    """
    # Get a list of all CSV files in the directory
    csv_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.csv')]

    # Read all CSV files into a Dask DataFrame
    ddf = dd.read_csv(csv_files, sep=';')
    
    return ddf


In [53]:
directory = '/data/leuven/351/vsc35102/scratch/Full Data Set/'
ddf = read_csv_files(directory)


In [57]:
#Inspect the data
ddf.head(npartitions=-1)


18062883

In [ ]:
#Inspect the length of the dask object 
len(ddf)

In [60]:
#Set the timestamp into the correct form and set the index to resutlt_timestapm 
ddf["result_timestamp"] = dd.to_datetime(ddf["result_timestamp"],
                                         dayfirst=True,
                                         format="%d/%m/%Y %H:%M:%S.%f"
                                        )
ddf_sorted = ddf.set_index('result_timestamp')

In [61]:
#Inspect the result 
ddf_sorted.head()

,#object_id,description,lamax,lamax_unit,laeq,laeq_unit,lceq,lceq_unit,lcpeak,lcpeak_unit
result_timestamp,,,,,,,,,,
2022-04-01 00:00:00.343,255443,MP 06: Parkstraat 2 La Filosovia,57.6,dB(A),51.3,dB(A),54.21,dB(C),67.1,dB(C)
2022-04-01 00:00:00.405,255444,MP 07: Naamsestraat 81,48.9,dB(A),46.5,dB(A),50.88,dB(C),65.18,dB(C)
2022-04-01 00:00:00.441,280324,MP08bis - Vrijthof,46,dB(A),44.2,dB(A),51.79,dB(C),63.62,dB(C)
2022-04-01 00:00:00.520,255439,MP 01: Naamsestraat 35 Maxim,62.5,dB(A),59.3,dB(A),64.56,dB(C),78.13,dB(C)
2022-04-01 00:00:00.885,255442,MP 05: Calvariekapel KU Leuven,55.8,dB(A),52.4,dB(A),55.38,dB(C),70.4,dB(C)


In [63]:
#Set the laeq column type to float for computations
ddf_sorted['laeq'] = ddf_sorted['laeq'].astype(float) 


In [64]:
#Group the data based on the location and cluster into 1min intervals 
#Kick of the computation and store the lazy dataframe into pandas 
def per_group(df):
    return (
        df
        .resample("1T")
        .agg({'laeq':'mean'})
    )

pandas3 = ddf_sorted.groupby("description").apply(per_group).compute()

/user/leuven/351/vsc35102/.local/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  # Remove the CWD from sys.path while we load stuff.


In [65]:
#Reset the index 
pandas4 = pandas3.reset_index()
pandas4.head(60)

,description,result_timestamp,laeq
0,MP 03: Naamsestraat 62 Taste,2022-04-01 00:00:00,53.640000
1,MP 03: Naamsestraat 62 Taste,2022-04-01 00:01:00,49.456667
2,MP 03: Naamsestraat 62 Taste,2022-04-01 00:02:00,51.306667
3,MP 03: Naamsestraat 62 Taste,2022-04-01 00:03:00,55.215000
4,MP 03: Naamsestraat 62 Taste,2022-04-01 00:04:00,62.083333
5,MP 03: Naamsestraat 62 Taste,2022-04-01 00:05:00,59.815000
6,MP 03: Naamsestraat 62 Taste,2022-04-01 00:06:00,62.120000
7,MP 03: Naamsestraat 62 Taste,2022-04-01 00:07:00,61.126667
8,MP 03: Naamsestraat 62 Taste,2022-04-01 00:08:00,53.910000
9,MP 03: Naamsestraat 62 Taste,2022-04-01 00:09:00,54.236667
